# Aprendizaje Automático - Intermedio

## Valores ausentes

### Introducción

En este tutorial, aprenderemos tres enfoques para tratar los valores ausentes. Luego compararemos la efectividad de estos enfoques en un conjunto de datos del mundo real.

Hay muchas formas en las que los datos pueden terminar con valores ausentes. Por ejemplo,

+ Una casa de 2 dormitorios no incluirá un valor para el tamaño de un tercer dormitorio.
+ Un encuestado puede optar por no compartir sus ingresos.

La mayoría de las librerías de aprendizaje automático (incluido scikit-learn) dan un error si intentamos construir un modelo utilizando datos con valores faltantes. Por lo tanto, deberemos elegir una de las estrategias a continuación.

### Tres enfoques

#### 1) Opción simple: Eliminar las columnas con valores faltantes

La opción más simple es eliminar las columnas con valores faltantes.

![Opción1](./images/valores_faltantes_1.png)

A menos que falten la mayoría de los valores en las columnas descartadas, el modelo pierde acceso a mucha información (¡potencialmente útil!) con este enfoque. Como ejemplo extremo, considere un conjunto de datos con 10.000 filas, donde a una columna importante le falta una sola entrada. ¡Este enfoque eliminaría la columna por completo!

#### 2) Una mejor opción: Imputación

La **imputación** completa los valores faltantes con algún número. Por ejemplo, podemos completar el valor medio a lo largo de cada columna.

![Valores_faltantes_2](./images/valores_faltantes_2.png)

El valor imputado no será exactamente correcto en la mayoría de los casos, pero generalmente conduce a modelos más precisos de los que obtendría al eliminar la columna por completo.

#### 3) Una extesión a la imputacion

La imputación es el enfoque estándar y generalmente funciona bien. Sin embargo, los valores imputados pueden estar sistemáticamente por encima o por debajo de sus valores reales (que no se recopilaron en el conjunto de datos). O las filas con valores ausentes pueden ser únicas de alguna otra manera. En ese caso, tu modelo haría mejores predicciones al considerar qué valores faltaban originalmente.

![valores_faltantes_3](./images/valores_faltantes_3.png)

En este enfoque imputamos los valores ausentes como antes. Y, además, para cada columna con entradas faltantes en el conjunto de datos original, agregamos una nueva columna que muestra la ubicación de las entradas imputadas.

En algunos casos, esto mejorará significativamente los resultados. En otros casos, no ayuda en absoluto.

### Ejemplo

En el ejemplo, trabajaremos con el conjunto de datos de [Melbourne Housing](https://www.kaggle.com/dansbecker/melbourne-housing-snapshot). Nuestro modelo utilizará información como la cantidad de habitaciones y el tamaño del terreno para predecir el precio de la vivienda.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Carga los data
data = pd.read_csv('./input/melbourne-housing-snapshot/melb_data.csv')

# Selecciona el objetivo
y = data.Price

# Para mantener las cosas simples, usaremos solo predictores numéricos
melb_predictors = data.drop(['Price'], axis=1)
X = melb_predictors.select_dtypes(exclude=['object'])

# divide los datos en entrenamiento y validación
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

#### Define la función para medir la calidad de cada enfoque

Definimos una función `score_dataset()` para comparar los diferentes enfoques de tratar los valores perdidos. Esta función informa el error absoluto medio (MAE) de un modelo de random forest.

In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Función para comparar diferentes enfoques
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

#### Puntuación del Enfoque 1 (Eliminar columnas con valores ausentes)

Como estamos trabajando con conjuntos de entrenamiento y validación, tenemos cuidado de eliminar las mismas columnas en ambos dataframes.

In [3]:
# Obtiene los nombres de las columnas con valores ausentes
cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()]

# Elimina las columnas en datos de entrenamiento y validación
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

print("MAE de Enfoque 1 (Eliminar columnas con valores ausentes):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE de Enfoque 1 (Eliminar columnas con valores ausentes):
183550.22137772635


#### Puntuación del Enfoque 2 (Imputación)

Usamos `SimpleImputer` para reemplazar los valores faltantes con el valor medio a lo largo de cada columna.

Aunque es simple, completar el valor medio generalmente funciona bastante bien (pero esto varía según el conjunto de datos). Si bien los estadísticos han experimentado formas más complejas de determinar los valores imputados (como la **imputación de regresión**, por ejemplo), las estrategias complejas generalmente no brindan ningún beneficio adicional.

In [4]:
from sklearn.impute import SimpleImputer

# Imputación
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# La imputación elimina los nombres de las columnas; los recuperamos
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print("MAE de Enfoque 2 (Imputación):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE de Enfoque 2 (Imputación):
178166.46269899711


Vemos que el Enfoque 2 tiene un MAE más bajo que el Enfoque 1, por lo que el Enfoque 2 se ejecutó mejor en este conjunto de datos.

#### Puntuación del Enfoque 3 (Una extensión a la Imputación)

A continuación, imputamos los valores ausentes al tiempo que hacemos un seguimiento de los valores imputados.

In [5]:
# Hacemos una copia para evitar cambiar los datos originales (cuando imputemos)
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

# Creamos nuevas columnas indicando que serán imputadas
for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()

# Imputación
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))

# La imputación elimina nombres de columnas; las recuperamos
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

print("MAE del Enfoque 3 (Una extesión a la Imputación):")
print(score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid))

MAE del Enfoque 3 (Una extesión a la Imputación):
178927.503183954


Como podemos ver, el Enfoque 3 tuvo un desempeño ligeramente peor que el Enfoque 2.

#### Entonces, ¿por qué la imputación funcionó mejor que eliminar las columnas?

Los datos de entrenamiento tienen 10864 filas y 12 columnas, donde tres columnas contienen datos faltantes. Para cada columna, faltan menos de la mitad de las entradas. Por lo tanto, eliminar las columnas elimina mucha información útil, por lo que tiene sentido que la imputación funcione mejor.

In [6]:
# Tamaño de los datos de entrenamiento (num_rows, num_columns)
print(X_train.shape)

# Número de valores ausentes en cada columna de los datos de entrenamiento
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(10864, 12)
Car               49
BuildingArea    5156
YearBuilt       4307
dtype: int64


### Conclusión

Como es común, la imputación de valores ausentes (en el **Enfoque 2** y el **Enfoque 3**) arrojó mejores resultados, en comparación a cuando simplemente eliminamos columnas con valores ausentes (en el **Enfoque 1**).

## Ejercicio

En este ejercicio trabajaremos con datos de [Housing Prices Competition for Kaggle Learn Users](https://www.kaggle.com/c/home-data-for-ml-course). 

![Ames Housing dataset image](https://i.imgur.com/lTJVG4e.png)

Vamos a cargar los conjuntos de entrenamiento y validación en `X_train`, `X_valid`, `y_train`, e `y_valid`.  Las pruebas son cargadas en `X_test`.

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Lee los datos
X_full = pd.read_csv('./input/train.csv', index_col='Id')
X_test_full = pd.read_csv('./input/test.csv', index_col='Id')

# Elimina las filas con objetivos ausentes, separa el objetivo de los predictores
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

# Para mantener las cosas simples solo usaremos predictores numéricos
X = X_full.select_dtypes(exclude=['object'])
X_test = X_test_full.select_dtypes(exclude=['object'])

# Separamos los datos de validación a partir de los datos de entrenamiento
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [8]:
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
619,20,90.0,11694,9,5,2007,2007,452.0,48,0,...,774,0,108,0,0,260,0,0,7,2007
871,20,60.0,6600,5,5,1962,1962,0.0,0,0,...,308,0,0,0,0,0,0,0,8,2009
93,30,80.0,13360,5,7,1921,2006,0.0,713,0,...,432,0,0,44,0,0,0,0,8,2009
818,20,NaN,13265,8,5,2002,2002,148.0,1218,0,...,857,150,59,0,0,0,0,0,7,2008
303,20,118.0,13704,7,5,2001,2002,150.0,0,0,...,843,468,81,0,0,0,0,0,1,2006


Ya podemos ver algunos valores ausentes en las primeras filas. En el siguiente paso, obtendremos una comprensión más completa de los valores faltantes en el conjunto de datos.

### Paso 1: Investigación preliminar

In [9]:
# Tamaño de los datos de entrenamiento (num_rows, num_columns)
print(X_train.shape)

# Número de valores ausentes de cada columna de los datos de entrenamiento
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(1168, 36)
LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64


**¿Cuántas filas tienen los datos de entrenamiento?**

In [10]:
num_rows = 1168

**¿Cuántas columnas en los datos de entrenamiento tienen valores faltantes?**

In [11]:
num_cols_with_missing = 3

**¿Cuántas entradas faltantes están contenidas en todos los datos de entrenamiento?**

In [12]:
tot_missing = 212 + 6 + 58

Teniendo en cuenta las respuestas anteriores, ¿cuál crees que es el mejor enfoque para tratar con los valores ausentes?

**Solución**: dado que hay relativamente pocas entradas faltantes en los datos (la columna con el mayor porcentaje de valores faltantes tiene menos del 20% de sus entradas), podemos esperar que sea poco probable que la eliminación de columnas arroje buenos resultados. Esto se debe a que estaríamos desechando una gran cantidad de datos valiosos, por lo que la imputación probablemente funcionará mejor.

Para comparar diferentes enfoques para tratar con valores perdidos, usaremos la misma función `score_dataset ()` del tutorial. Esta función informa el [error absoluto medio](https://en.wikipedia.org/wiki/Mean_absolute_error) (MAE) de un modelo de random forest.

In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Función para comparar diferentes enfoques
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

### Paso 2: Eliminar columnas con valores ausentes

En este paso, preprocesaremos los datos en `X_train` y `X_valid` para eliminar columnas con valores faltantes. Establece los DataFrames preprocesados a `reduce_X_train` y `reduce_X_valid`, respectivamente.

In [14]:
# Obtenemos los nombres de las columnas con valores ausentes
cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()]

# Eliminamos las columnas en los datos de entrenamiento y validación
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

In [15]:
print("MAE (Eliminar columnas con valores ausentes):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE (Eliminar columnas con valores ausentes):
17837.82570776256


### Paso 3: Imputación

En este paso imputaremos los valores faltantes con el valor medio de cada columna. Establece los DataFrames preprocesados en `imputed_X_train` y `imputed_X_valid`. Asegúrate de que los nombres de las columnas coincidan con los de `X_train` y `X_valid`.

In [16]:
from sklearn.impute import SimpleImputer

# Imputación
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# La imputación elimina los nombres de las columnas; los recuperamos
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

In [17]:
print("MAE (Imputación):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE (Imputación):
18062.894611872147


Comparemos el MAE de cada enfoque. ¿Te sorprende algo de los resultados? ¿Por qué crees que un enfoque funcionó mejor que el otro?

**Solución**: Dado que hay tan pocos valores faltantes en el conjunto de datos, esperaríamos que la imputación funcione mejor que eliminar columnas por completo. Sin embargo, ¡vemos que eliminar columnas funciona un poco mejor! Si bien esto probablemente se puede atribuir parcialmente al ruido en el conjunto de datos, otra explicación potencial es que el método de imputación no es una gran elección con este conjunto de datos. Es decir, tal vez en lugar de completar el valor medio, tiene más sentido establecer cada valor faltante en un valor de 0, completar con el valor más frecuente o usar algún otro método. Por ejemplo, consideremos la columna `GarageYrBlt` (que indica el año en que se construyó el garaje). Es probable que en algunos casos, un valor faltante pueda indicar una casa que no tiene garaje. ¿Tiene más sentido completar el valor medio de cada columna en este caso? ¿O podríamos obtener mejores resultados al completar el valor mínimo de cada columna? No está del todo claro qué es lo mejor en este caso, pero tal vez podamos descartar algunas opciones de inmediato; por ejemplo, establecer valores faltantes en esta columna a 0 probablemente arrojará resultados horribles.

### Paso 4: Generar predicciones de prueba

En este paso final, utilizaremos cualquier enfoque que elijamos para manejar los valores faltantes. Una vez que hayamos procesado previamente las características de entrenamiento y validación, entrenaremos y evaluaremos un modelo random forest. Luego, ¡preprocesaremos los datos de prueba antes de generar predicciones que puedan enviarse a la competición!

Vamos a preprocesar los datos de entrenamiento y validación. Establece los DataFrames preprocesados en `final_X_train` y `final_X_valid`. **¡Puedes utilizar cualquier enfoque que elija aquí!**, solo necesitas asegurarte de que:
- los DataFrames preprocesados tienen el mismo número de columnas,
- los DataFrames preprocesados no tienen valores faltantes,
- `final_X_train` y `y_train` tienen el mismo número de filas, y
- `final_X_valid` e `y_valid` tienen el mismo número de filas.

In [20]:
# Imputation
final_imputer = SimpleImputer(strategy='median')
final_X_train = pd.DataFrame(final_imputer.fit_transform(X_train))
final_X_valid = pd.DataFrame(final_imputer.transform(X_valid))

final_X_train.columns = X_train.columns
final_X_valid.columns = X_valid.columns

In [21]:
# Define y entrena el modelo
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(final_X_train, y_train)

# Obtiene las predicciones de validación y el MAE
preds_valid = model.predict(final_X_valid)
print("MAE (Nuestro enfoque):")
print(mean_absolute_error(y_valid, preds_valid))

MAE (Nuestro enfoque):
17791.59899543379


Vamos a preprocesar los datos de prueba. Asegúrate de utilizar un método que coincida con la forma en que se procesaron previamente los datos de entrenamiento y validación, y configura las funciones de prueba preprocesadas en `final_X_test`.

Luego usa las características de prueba preprocesadas y el modelo entrenado para generar predicciones de prueba en `preds_test`.

Asegúrate de:
- el DataFrame de prueba preprocesado no tiene valores faltantes, y
- `final_X_test` tiene el mismo número de filas que` X_test`.

In [22]:
# Preprocesa los datos de prueba
final_X_test = pd.DataFrame(final_imputer.transform(X_test))

# Obtiene las predicciones de prueba
preds_test = model.predict(final_X_test)

Guardamos los resultados en un archivo CSV que se puede enviar directamente a la competición.

In [23]:
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('./output/submission.csv', index=False)